In [18]:
import duckdb

In [25]:
duckdb.execute("CREATE TABLE IF NOT EXISTS trips_2024 AS SELECT * FROM '../../data/bicis-ba/trips_2024.csv'")
duckdb.sql("SELECT * FROM trips_2024 LIMIT 10").df()

,Id_recorrido,duracion_recorrido,fecha_origen_recorrido,id_estacion_origen,nombre_estacion_origen,direccion_estacion_origen,long_estacion_origen,lat_estacion_origen,fecha_destino_recorrido,id_estacion_destino,nombre_estacion_destino,direccion_estacion_destino,long_estacion_destino,lat_estacion_destino,id_usuario,modelo_bicicleta,género
0,20610093BAEcobici,858,2024-02-08 20:58:32,516BAEcobici,368 - PLAZA DE LA SHOÁ,Av. Del Libertador 3899,-58.423347,-34.570998,2024-02-08 21:12:50,308BAEcobici,251 - Solar de la Abadía,Maure 1819,-58.436752,-34.567633,85441BAEcobici,ICONIC,MALE
1,21419508BAEcobici,512,2024-04-17 08:37:36,252BAEcobici,331 - CALIFORNIA,California 1774 y General Hornos,-58.374336,-34.647121,2024-04-17 08:46:08,73BAEcobici,073 - Ruy Díaz de Guzmán,Avenida Martin Garcia y Ruy Díaz de Guzmán,-58.371823,-34.630681,512851BAEcobici,ICONIC,MALE
2,21421689BAEcobici,964,2024-04-17 11:38:10,277BAEcobici,292 - PLAZA BOLIVIA,Olleros Av. & Del Libertador Av.,-58.436115,-34.563539,2024-04-17 11:54:14,322BAEcobici,289 - MONTAÑESES,2786 Montañeses,-58.454181,-34.551304,27717BAEcobici,ICONIC,FEMALE
3,21436007BAEcobici,564,2024-04-18 09:25:38,66BAEcobici,066 - Billinghurst,3508 Carcova,-58.413871,-34.594547,2024-04-18 09:35:02,124BAEcobici,124 - UGARTECHE,3050 Ugarteche,-58.411965,-34.580538,870951BAEcobici,ICONIC,MALE
4,21286197BAEcobici,"1,145",2024-04-05 19:56:39,202BAEcobici,001 - FACULTAD DE DERECHO,Av. Pres. Figueroa Alcorta 2120,-58.390602,-34.583749,2024-04-05 20:15:44,128BAEcobici,128 - PARQUE DEL BAJO,Tte. Gral. Juan Domingo Perón 133,-58.368821,-34.605152,23367BAEcobici,ICONIC,None
5,20504774BAEcobici,"1,593",2024-01-30 02:38:38,478BAEcobici,015 - AMENABAR,Teodoro García 2613,-58.447524,-34.572056,2024-01-30 03:05:11,384BAEcobici,267 - Plaza 25 De Agosto,14 de julio de 1194,-58.461835,-34.579519,796520BAEcobici,ICONIC,FEMALE
6,21162330BAEcobici,"1,614",2024-03-26 01:20:40,524BAEcobici,338 - AV. CABILDO,Av. Cabildo 2586,-58.460703,-34.557933,2024-03-26 01:47:34,524BAEcobici,338 - AV. CABILDO,Av. Cabildo 2586,-58.460703,-34.557933,796520BAEcobici,ICONIC,FEMALE
7,21886942BAEcobici,768,2024-05-30 18:19:37,165BAEcobici,165 - PLAZA MONSEÑOR MIGUEL DE ANDREA,Dr Tomas Manuel De Anchorena & Jose A. Cabrera,-58.407614,-34.597048,2024-05-30 18:32:25,99BAEcobici,099 - Malabia,780 Malabia & Loyola,-58.435408,-34.596096,70653BAEcobici,ICONIC,FEMALE
8,21153643BAEcobici,"1,115",2024-03-25 15:23:42,316BAEcobici,205 - SAN MARTIN,1471 Cucha Cucha,-58.453475,-34.605567,2024-03-25 15:42:17,281BAEcobici,350 - Plaza Irlanda,Av. Gaona 2200,-58.458315,-34.613778,1084074BAEcobici,ICONIC,FEMALE
9,21059870BAEcobici,732,2024-03-18 06:49:29,132BAEcobici,132 - CORRIENTES,Reconquista & Corrientes Av.,-58.372763,-34.603369,2024-03-18 07:01:41,228BAEcobici,010 - Madero UCA,"1600 Moreau De Justo, Alicia Av.",-58.365668,-34.616488,831544BAEcobici,ICONIC,MALE


In [31]:
duckdb.execute('DROP TABLE IF EXISTS estaciones')
duckdb.execute("""
CREATE TABLE estaciones (
    id INTEGER,
    nombre VARCHAR,
    latitud FLOAT,
    longitud FLOAT,
    direccion VARCHAR
) 
""")
duckdb.execute("""
INSERT INTO estaciones
SELECT DISTINCT CAST(REPLACE(id, 'BAEcobici', '') AS INTEGER) AS id, nombre, latitud, longitud, direccion
FROM (
    SELECT id_estacion_destino AS id, nombre_estacion_destino AS nombre, lat_estacion_destino AS latitud, long_estacion_destino AS longitud, direccion_estacion_destino AS direccion
    FROM trips_2024
    UNION
    SELECT id_estacion_origen AS id, nombre_estacion_origen AS nombre, lat_estacion_origen AS latitud, long_estacion_origen AS longitud, direccion_estacion_origen AS direccion
    FROM trips_2024
) AS subquery
""")
duckdb.sql("SELECT * FROM estaciones").df()


,id,nombre,latitud,longitud,direccion
0,121,121 - YATAY,-34.601173,-58.428509,925 Yatay& Humahuaca
1,506,040 - PARQUE SALGUERO,-34.569435,-58.399368,Av. Costa Salguero 1222
2,516,368 - PLAZA DE LA SHOÁ,-34.570999,-58.423347,Av. Del Libertador 3899
3,379,390 - De Los Ombues,-34.560200,-58.428101,Av. de los Ombúes 300
4,213,136 - Acevedo,-34.599659,-58.442684,Acevedo y Padilla
...,...,...,...,...,...
350,63,063 - Reconquista,-34.599777,-58.374687,Viamonte 555
351,381,274 - Gavilán,-34.616245,-58.465885,Gavilán 1200
352,117,117 - HUMBERTO 1°,-34.619553,-58.373158,Av. Carlos Calvo 513
353,425,394 - Av. La Plata,-34.643013,-58.422844,Av. La Plata 2400


In [33]:
# Now we drop redundant columns from trips_2024
duckdb.execute("ALTER TABLE trips_2024 DROP COLUMN IF EXISTS nombre_estacion_origen")
duckdb.execute("ALTER TABLE trips_2024 DROP COLUMN  IF EXISTS lat_estacion_origen")
duckdb.execute("ALTER TABLE trips_2024 DROP COLUMN  IF EXISTS long_estacion_origen")
duckdb.execute("ALTER TABLE trips_2024 DROP COLUMN  IF EXISTS direccion_estacion_origen")
duckdb.execute("ALTER TABLE trips_2024 DROP COLUMN  IF EXISTS nombre_estacion_destino")
duckdb.execute("ALTER TABLE trips_2024 DROP COLUMN  IF EXISTS lat_estacion_destino")
duckdb.execute("ALTER TABLE trips_2024 DROP COLUMN  IF EXISTS long_estacion_destino")
duckdb.execute("ALTER TABLE trips_2024 DROP COLUMN  IF EXISTS direccion_estacion_destino")
